In [20]:
## import data manipulation packages
import pandas as pd
import numpy as np
from sklearn.neighbors import DistanceMetric
from math import radians

## import dataset "medium size" as variable 'city' and drop NaN
cities = pd.read_excel('data/worldcities.xlsx')
ct = cities.dropna(axis = 'rows', how = 'any')
## considero solo al di sopra dell'equatore
## ct = ct[ct.lat >= 0]

In [21]:
#add london Start (spostando lievemente la longitudine, così che sia più ad est di London)
London_st = ct.loc[(ct['city'] == 'London') & (ct['iso3'] == 'GBR')]
London_st['city']='London_st' 
London_st['lng'] = London_st['lng'] + 0.2

<ipython-input-21-b81702dafba4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  London_st['city']='London_st'
<ipython-input-21-b81702dafba4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  London_st['lng'] = London_st['lng'] + 0.2


In [22]:
## attacco la riga London_st al dataframe
ct = ct.append(London_st)

In [23]:
## resetto l'index, che sia progressivo
ct = ct.reset_index()


In [24]:
## concatenate iso2 and city to get unique id
ct['ID'] = ct['city'].map(str) + ct['iso2'].map(str)

In [25]:
## drop not usable columns
ct = ct.drop(['city_ascii', 'country', 'iso2', 'admin_name', 'capital', 'id'], axis = 1)

In [26]:
## droppo la colonna index duplicata
ct = ct.drop('index', axis = 1)

In [27]:
# population - dove la popolazione è > di 200,000 abitanti aggiungo peso 2
pop = np.where(ct['population'] < 200000 , 0 , 2 )
#same state  - dove lo stato è diverso aggiungo peso 2
i = ct['iso3'].to_numpy()
st = (i[:, None ] != i) * 2

In [28]:
# direction - lavoro sulla longitudine e prendo le longitudini ad "est", normalizzando le long negative aggiungendo 150
## forse dobbiamo aggiungere 180? visto che dovrebbe essere su 360 totale?
dr_x = np.where(ct['lng']>= 0 , ct['lng'] , (ct['lng'] + 180) + 180)
x = dr_x

dr= (x[:, None ] < x)*1
rang= (x[: , None ] < x+10 )*1

In [29]:
# metto i giusti indici alle matrici 
direction = pd.DataFrame(dr*rang.T, columns = ct['ID'], index = ct['ID'])
same_state = pd.DataFrame(st, columns = ct['ID'], index = ct['ID'])
population = pd.DataFrame(pop , index = ct['ID'])

In [30]:
direction

ID,TokyoJP,JakartaID,DelhiIN,MumbaiIN,ManilaPH,ShanghaiCN,São PauloBR,SeoulKR,Mexico CityMX,GuangzhouCN,...,RodeoAR,GastreAR,TinogastaAR,TelsenAR,GreytownNI,SayḩūtYE,Río CuartoAR,La EsmeraldaVE,HödrögöMN,London_stGB
ID,,,,,,,,,,,,,,,,,,,,,
TokyoJP,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
JakartaID,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
DelhiIN,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MumbaiIN,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ManilaPH,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SayḩūtYE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Río CuartoAR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
La EsmeraldaVE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [31]:
## coversion of coordinated in degrees to radians
ct['lat'] = np.radians(ct['lat'])
ct['lng'] = np.radians(ct['lng'])

In [32]:
## get the scipy metrics
dist = DistanceMetric.get_metric('haversine')

In [33]:
## prepare the array to get in input for the distance calculation
#non credo che serva 

#ct[['lat','lng']].to_numpy()

In [34]:
## pass pairwise function to get the distance / multiply for 6373 as earth sperical radius is 6373kms
#dist.pairwise(ct [['lat','lng']].to_numpy())*6373

In [35]:
#cerchiamo di avere meno dati 
D = np.where(  direction > 0  , dist.pairwise(ct [['lat','lng']].to_numpy())*6373 , 0   )
## create the distance matrix with cities in the indexes
distance = pd.DataFrame(D, columns = ct['ID'], index = ct['ID'])

In [36]:
from pandas import DataFrame

In [37]:
## popolare per criteri (più vicino +2, secondo più vicino +4, terzo più vicino +8, il resto 'no go')
dis = distance.T.replace(0, 0)
dis = dis.replace(dis.apply(lambda x: x[x > 0].min(axis=0)), 2)
dis = dis.replace(dis.apply(lambda x: x[x > 2].min(axis=0)), 4)
dis = dis.replace(dis.apply(lambda x: x[x > 4].min(axis=0)), 8)
dis = dis.where((dis <= 8), 0) 
dis

ID,TokyoJP,JakartaID,DelhiIN,MumbaiIN,ManilaPH,ShanghaiCN,São PauloBR,SeoulKR,Mexico CityMX,GuangzhouCN,...,RodeoAR,GastreAR,TinogastaAR,TelsenAR,GreytownNI,SayḩūtYE,Río CuartoAR,La EsmeraldaVE,HödrögöMN,London_stGB
ID,,,,,,,,,,,,,,,,,,,,,
TokyoJP,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
JakartaID,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
DelhiIN,0.0,0.0,0.0,1166.997459,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
MumbaiIN,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
ManilaPH,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,1248.62455,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SayḩūtYE,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
Río CuartoAR,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,...,557.012514,1103.133020,641.220957,1054.985097,0.0,0.0,0.0,4039.257048,0.0,0.0
La EsmeraldaVE,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,...,3733.495868,5067.878714,3481.586365,5069.283104,0.0,0.0,0.0,0.000000,0.0,0.0


In [19]:
graph =((dis + same_state + pop.T) * dis/dis)
graph = graph.where((graph > 1), 0) 
    
graph

ID,TokyoJP,JakartaID,DelhiIN,MumbaiIN,ManilaPH,ShanghaiCN,São PauloBR,SeoulKR,Mexico CityMX,GuangzhouCN,...,RodeoAR,GastreAR,TinogastaAR,TelsenAR,GreytownNI,SayḩūtYE,Río CuartoAR,La EsmeraldaVE,HödrögöMN,London_stGB
ID,,,,,,,,,,,,,,,,,,,,,
TokyoJP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JakartaID,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DelhiIN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MumbaiIN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ManilaPH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SayḩūtYE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Río CuartoAR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
La EsmeraldaVE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
